In [1]:
import numpy as np
import pandas as pd

In [2]:
HF_weather = pd.read_csv('../data/HF_weather/raw/hf001-08-hourly-m.csv', parse_dates=['datetime'])

/home/steffen/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,5,7,9,11,13,15,17,19,21,23,25,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def get_gdd(data, stn_id, tbase=5, datetime="datetime", airtemp="airt"):
    """Calculate cumulative growing degree days (GDD) from weather station data

    Parameters
    ----------
    data : DataFrame
        Weather station data.
    stn_id : str
        Name of weather station where data is from.
    tbase : int, optional
        GDD base temperature, by default 5
    datetime : str, optional
        Name of datetime column in 'data', by default "datetime"
    airtemp : str, optional
        Name of air temperature column in 'data', by default "airt"

    Returns
    -------
    DataFrame
        Table containing mean daily air temperature and cumulative GDD (resets
        on January 1st each year).  One row per day.
    """
    gdd_df = (
        data[[datetime, airtemp]]
        .groupby(pd.Grouper(key=datetime, freq="1D"))
        .mean()
        .reset_index()
    )
    gdd_df.rename(columns={airtemp: "mean_airt"}, inplace=True)
    gdd_df["GDD"] = gdd_df[["mean_airt"]].applymap(
        lambda x: 0 if x <= tbase else x - tbase
    )

    data_datetime = pd.DatetimeIndex(gdd_df[datetime])

    for year in data_datetime.year.unique():
        gdd_df.loc[(data_datetime.year == year), "cumGDD"] = gdd_df[
            data_datetime.year == year
        ]["GDD"].cumsum()

    gdd_df["STN"] = stn_id

    return gdd_df

In [4]:
def get_frthw(data, stn_id, threshold=0, datetime="datetime", airtemp="airt"):
    """Calculate cumulative freeze-thaw cycles from weather station data.

    Parameters
    ----------
    data : DataFrame
        Weather station data.
    stn_id : str
        Name of weather station where data is from.
    threshold : int, optional
        Freeze-thaw temperature threshold, by default 0
    datetime : str, optional
        Name of datetime column in 'data', by default "datetime"
    airtemp : str, optional
        Name of air temperature column in 'data', by default "airt"

    Returns
    -------
    DataFrame
        Table containing cumulative freeze-thaw cycles (resets on
        January 1st each year).  One row per day.
    """

    # Initialize summary dataframe
    frthw_df = pd.DataFrame()

    data_datetime = pd.DatetimeIndex(data[datetime])
    data["frthw"] = np.nan

    for year in data_datetime.year.unique():

        year_data = data.loc[
            (data_datetime.year == year), [datetime, airtemp, "frthw"]
        ]
        frthw = (
            0.5 if year_data.iloc[0, 1] > threshold else 0
        )  # If first temp reading is above threshold, start frthw at 0.5

        for obs in range(year_data.shape[0]):
            if frthw % 1 == 0:
                if year_data.iloc[obs, 1] >= threshold:
                    frthw += 0.5
            else:
                if year_data.iloc[obs, 1] < threshold:
                    frthw += 0.5
            year_data.iloc[obs, 2] = frthw

        year_data = year_data.drop(columns=[airtemp])
        year_data = year_data.set_index(datetime).resample("1D").last()
        frthw_df = frthw_df.append(year_data)

    frthw_df["STN"] = stn_id

    return frthw_df

In [5]:
tbase = 5  # Growing degree day base temperature
gdd = get_gdd(HF_weather, "HF001", tbase=tbase)

threshold = 3  # Freeze-thaw temperature threshold
frzthw = get_frthw(HF_weather, "HF001", threshold=threshold)

gdd_frthw = pd.concat([gdd, frzthw.reset_index()["frthw"]], axis=1)

In [6]:
gdd_frthw.to_pickle('../data/HF_weather/gdd_frthw')